In [1]:
!pip install spotipy 

     |████████████████████████████████| 138 kB 3.3 MB/s eta 0:00:01
     |████████████████████████████████| 62 kB 1.0 MB/s  eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.10
    Uninstalling urllib3-1.25.10:
      Successfully uninstalled urllib3-1.25.10
  Attempting uninstall: requests
    Found existing installation: requests 2.24.0
    Uninstalling requests-2.24.0:
      Successfully uninstalled requests-2.24.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

botocore 1.18.16 requires urllib3<1.26,>=1.20, but you'll have urllib3 1.26.7 which is incompatible.


In [78]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import config
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests
import os 
import re

sp = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials(config.spotify['client_id'],
                                                                        config.spotify['client_secret']))

In [52]:
url = 'https://open.spotify.com/album/43otFXrY0bgaq5fB3GrZj6?si=_GGcUScGTK-P2LsvPnk2yQ&dl_branch=1'
uri = 'spotify:album:43otFXrY0bgaq5fB3GrZj6'

In [53]:
def get_album_songs(uri_info):
    uri = []
    track = []
    duration = []
    explicit = []
    track_number = []
    one = sp.album_tracks(uri_info, limit=50, offset=0, market='US')
    df1 = pd.DataFrame(one)
    
    for i, x in df1['items'].items():
        uri.append(x['uri'])
        track.append(x['name'])
        duration.append(x['duration_ms'])
        explicit.append(x['explicit'])
        track_number.append(x['track_number'])
    
    df2 = pd.DataFrame({
    'uri':uri,
    'track':track,
    'duration_ms':duration,
    'explicit':explicit,
    'track_number':track_number})
    
    return df2

In [54]:
album = get_album_songs(uri)
album

,uri,track,duration_ms,explicit,track_number
0,spotify:track:3bIQIx7hveYPQDdhjZ1kcq,The Getaway,250386,False,1
1,spotify:track:2oaK4JLVnmRGIO9ytBE1bt,Dark Necessities,302000,False,2
2,spotify:track:0cv2LgkvEoQiGgFWcZaAMA,We Turn Red,200466,False,3
3,spotify:track:0pjCkLjbgSLn5c0Ilwuv8z,The Longest Wave,211520,False,4
4,spotify:track:2XTkpF9T2PKvcLgamGJGx1,Goodbye Angels,268733,False,5
5,spotify:track:6GsP3uMCd0Dn5T37C93waZ,Sick Love,221440,False,6
6,spotify:track:5m578MgxuEC956dulprCe8,Go Robot,263800,False,7
7,spotify:track:0SMzXgUFqcAt0WNCsGm3tM,Feasting on the Flowers,202933,False,8
8,spotify:track:5y7IfSDhBHFVnlGj1UrlA1,Detroit,226640,False,9
9,spotify:track:3iXNy7VaY9OqJFBwzhmbOh,This Ticonderoga,215200,False,10


In [55]:
def get_track_info(df):
    danceability = []
    energy = []
    key = []
    loudness = []
    speechiness = []
    acousticness = []
    instrumentalness = []
    liveness = []
    valence = []
    tempo = []
    
    for i in df['uri']:
        for x in sp.audio_features(tracks=[i]):
            danceability.append(x['danceability'])
            energy.append(x['energy'])
            key.append(x['key'])
            loudness.append(x['loudness'])
            speechiness.append(x['speechiness'])
            acousticness.append(x['acousticness'])
            instrumentalness.append(x['instrumentalness'])
            liveness.append(x['liveness'])
            valence.append(x['valence'])
            tempo.append(x['tempo'])
            
    df2 = pd.DataFrame({
    'danceability':danceability,
    'energy':energy,
    'key':key,
    'loudness':loudness,
    'speechiness':speechiness,
    'acousticness':acousticness,
    'instrumentalness':instrumentalness,
    'liveness':liveness,
    'valence':valence,
    'tempo':tempo})
    
    return df2

In [56]:
song_info = get_track_info(album)
song_info

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.797,0.642,0,-8.082,0.0725,0.079900,0.000012,0.3710,0.667,113.999
1,0.700,0.742,5,-6.777,0.0716,0.072200,0.019900,0.1100,0.197,91.959
2,0.610,0.872,2,-5.428,0.0422,0.116000,0.147000,0.1520,0.842,164.038
3,0.476,0.511,8,-6.250,0.0324,0.319000,0.000129,0.1190,0.339,148.158
4,0.365,0.804,9,-5.922,0.1050,0.104000,0.000066,0.1500,0.577,171.597
5,0.591,0.794,4,-5.935,0.0690,0.183000,0.000000,0.2920,0.875,147.974
6,0.852,0.651,8,-6.254,0.0411,0.043200,0.019200,0.1580,0.960,131.010
7,0.670,0.750,6,-5.502,0.0583,0.137000,0.000004,0.1970,0.880,82.041
8,0.439,0.885,4,-4.679,0.0625,0.000277,0.003670,0.3720,0.495,92.738
9,0.238,0.949,8,-4.575,0.2470,0.016000,0.000000,0.0864,0.405,87.843


In [57]:
the_getaway = album.merge(song_info, left_index=True, right_index=True)
the_getaway

,uri,track,duration_ms,explicit,track_number,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,spotify:track:3bIQIx7hveYPQDdhjZ1kcq,The Getaway,250386,False,1,0.797,0.642,0,-8.082,0.0725,0.079900,0.000012,0.3710,0.667,113.999
1,spotify:track:2oaK4JLVnmRGIO9ytBE1bt,Dark Necessities,302000,False,2,0.700,0.742,5,-6.777,0.0716,0.072200,0.019900,0.1100,0.197,91.959
2,spotify:track:0cv2LgkvEoQiGgFWcZaAMA,We Turn Red,200466,False,3,0.610,0.872,2,-5.428,0.0422,0.116000,0.147000,0.1520,0.842,164.038
3,spotify:track:0pjCkLjbgSLn5c0Ilwuv8z,The Longest Wave,211520,False,4,0.476,0.511,8,-6.250,0.0324,0.319000,0.000129,0.1190,0.339,148.158
4,spotify:track:2XTkpF9T2PKvcLgamGJGx1,Goodbye Angels,268733,False,5,0.365,0.804,9,-5.922,0.1050,0.104000,0.000066,0.1500,0.577,171.597
5,spotify:track:6GsP3uMCd0Dn5T37C93waZ,Sick Love,221440,False,6,0.591,0.794,4,-5.935,0.0690,0.183000,0.000000,0.2920,0.875,147.974
6,spotify:track:5m578MgxuEC956dulprCe8,Go Robot,263800,False,7,0.852,0.651,8,-6.254,0.0411,0.043200,0.019200,0.1580,0.960,131.010
7,spotify:track:0SMzXgUFqcAt0WNCsGm3tM,Feasting on the Flowers,202933,False,8,0.670,0.750,6,-5.502,0.0583,0.137000,0.000004,0.1970,0.880,82.041
8,spotify:track:5y7IfSDhBHFVnlGj1UrlA1,Detroit,226640,False,9,0.439,0.885,4,-4.679,0.0625,0.000277,0.003670,0.3720,0.495,92.738
9,spotify:track:3iXNy7VaY9OqJFBwzhmbOh,This Ticonderoga,215200,False,10,0.238,0.949,8,-4.575,0.2470,0.016000,0.000000,0.0864,0.405,87.843


chose this album because was ranked #1 in 2016 

In [58]:
url = 'https://open.spotify.com/album/2Y9IRtehByVkegoD7TcLfi?si=AcMML6WLQaK1In7epcJGrA&dl_branch=1'
uri = 'spotify:album:2Y9IRtehByVkegoD7TcLfi'

In [59]:
album2 = get_album_songs(uri)
album2

,uri,track,duration_ms,explicit,track_number
0,spotify:track:0aGQHMr7bc23Y9Ts84ffop,Around the World,238733,False,1
1,spotify:track:1Se0r96r0gnqg67kJPmESc,Parallel Universe,269360,False,2
2,spotify:track:1G391cbiT3v3Cywg8T7DM1,Scar Tissue,215906,False,3
3,spotify:track:64BbK9SFKH2jk86U3dGj2P,Otherside,255373,False,4
4,spotify:track:0GcWf1Cc4ixN35D3O9VdcP,Get on Top,198066,True,5
5,spotify:track:48UPSzbZjgc449aqz8bxox,Californication,329733,False,6
6,spotify:track:3QJnjJH53Ocg22MaLutrgJ,Easily,231333,False,7
7,spotify:track:6WUjz18yIeTGDR1E69WHtf,Porcelain,163706,False,8
8,spotify:track:2xDHUsWEsH33sKbFqDZPFj,Emit Remmus,240200,False,9
9,spotify:track:2bo1tTjK87OHKKUDFthqHx,I Like Dirt,157626,False,10


In [60]:
song2 = get_track_info(album2)
song2

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,0.504,0.960,0,-1.464,0.1830,0.004170,0.569000,0.0613,0.7740,96.682
1,0.343,0.983,0,-2.359,0.2120,0.001070,0.296000,0.0708,0.1860,124.536
2,0.595,0.717,0,-4.803,0.0295,0.077900,0.002740,0.1080,0.5470,88.969
3,0.458,0.795,0,-3.265,0.0574,0.003160,0.000202,0.0756,0.5130,123.229
4,0.573,0.994,4,-0.407,0.3420,0.028100,0.005430,0.3670,0.4160,114.563
5,0.592,0.767,9,-2.788,0.0270,0.002100,0.001650,0.1270,0.3280,96.483
6,0.283,0.982,0,-0.320,0.0703,0.000204,0.075600,0.0627,0.4380,124.149
7,0.371,0.259,7,-11.352,0.0312,0.443000,0.003840,0.0752,0.0467,140.425
8,0.354,0.964,5,-1.822,0.0973,0.000051,0.203000,0.3260,0.5260,113.079
9,0.712,0.969,4,-1.503,0.1610,0.011300,0.031900,0.0904,0.8020,105.077


In [61]:
californication = album2.merge(song2, left_index=True, right_index=True)
californication

,uri,track,duration_ms,explicit,track_number,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,spotify:track:0aGQHMr7bc23Y9Ts84ffop,Around the World,238733,False,1,0.504,0.960,0,-1.464,0.1830,0.004170,0.569000,0.0613,0.7740,96.682
1,spotify:track:1Se0r96r0gnqg67kJPmESc,Parallel Universe,269360,False,2,0.343,0.983,0,-2.359,0.2120,0.001070,0.296000,0.0708,0.1860,124.536
2,spotify:track:1G391cbiT3v3Cywg8T7DM1,Scar Tissue,215906,False,3,0.595,0.717,0,-4.803,0.0295,0.077900,0.002740,0.1080,0.5470,88.969
3,spotify:track:64BbK9SFKH2jk86U3dGj2P,Otherside,255373,False,4,0.458,0.795,0,-3.265,0.0574,0.003160,0.000202,0.0756,0.5130,123.229
4,spotify:track:0GcWf1Cc4ixN35D3O9VdcP,Get on Top,198066,True,5,0.573,0.994,4,-0.407,0.3420,0.028100,0.005430,0.3670,0.4160,114.563
5,spotify:track:48UPSzbZjgc449aqz8bxox,Californication,329733,False,6,0.592,0.767,9,-2.788,0.0270,0.002100,0.001650,0.1270,0.3280,96.483
6,spotify:track:3QJnjJH53Ocg22MaLutrgJ,Easily,231333,False,7,0.283,0.982,0,-0.320,0.0703,0.000204,0.075600,0.0627,0.4380,124.149
7,spotify:track:6WUjz18yIeTGDR1E69WHtf,Porcelain,163706,False,8,0.371,0.259,7,-11.352,0.0312,0.443000,0.003840,0.0752,0.0467,140.425
8,spotify:track:2xDHUsWEsH33sKbFqDZPFj,Emit Remmus,240200,False,9,0.354,0.964,5,-1.822,0.0973,0.000051,0.203000,0.3260,0.5260,113.079
9,spotify:track:2bo1tTjK87OHKKUDFthqHx,I Like Dirt,157626,False,10,0.712,0.969,4,-1.503,0.1610,0.011300,0.031900,0.0904,0.8020,105.077


chose this album because it's their highest grossing album - ranked #3 in 1999

In [75]:
page = requests.get('https://genius.com/Red-hot-chili-peppers-around-the-world-lyrics')
soup = BeautifulSoup(page.text, 'html.parser')
soup.prettify()

'<!DOCTYPE html>\n<html>\n <head>\n  <title>\n   Red\xa0Hot Chili Peppers – Around the World Lyrics | Genius Lyrics\n  </title>\n  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>\n  <meta content="width=device-width,initial-scale=1" name="viewport"/>\n  <meta content="app-id=709482991" name="apple-itunes-app"/>\n  <link href="https://assets.genius.com/images/apple-touch-icon.png?1632499442" rel="apple-touch-icon"/>\n  <link href="https://assets.genius.com/images/apple-touch-icon.png?1632499442" rel="apple-touch-icon"/>\n  <!-- Mobile IE allows us to activate ClearType technology for smoothing fonts for easy reading -->\n  <meta content="on" http-equiv="cleartype"/>\n  <meta content="f63347d284f184b0" name="y_key"/>\n  <meta content="Genius" property="og:site_name">\n   <meta content="265539304824" property="fb:app_id">\n    <meta content="308252472676410" property="fb:pages">\n     <link href="https://genius.com/opensearch.xml" rel="search" title="Genius" type="app

In [76]:
lyrics = soup.find('div', class_='Lyrics__Container-sc-1ynbvzw-2 jgQsqn')
lyrics.get_text()

AttributeError: 'NoneType' object has no attribute 'get_text'

In [71]:
def scrape_lyrics(artistname, songname):
    artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
    songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
    page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics1 = html.find("div", class_="lyrics")
    lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-2 jgQsqn")
    if lyrics1:
        lyrics = lyrics1.get_text()
    elif lyrics2:
        lyrics = lyrics2.get_text()
    elif lyrics1 == lyrics2 == None:
        lyrics = None
    return lyrics

#function to attach lyrics onto data frame
#artist_name should be inserted as a string
def lyrics_onto_frame(df1, artist_name):
    for i,x in enumerate(df1['track']):
        test = scrape_lyrics(artist_name, x)
        df1.loc[i, 'lyrics'] = test
    return df1

In [72]:
lyrics_onto_frame(the_getaway, 'Red hot chili peppers')

,uri,track,duration_ms,explicit,track_number,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,lyrics
0,spotify:track:3bIQIx7hveYPQDdhjZ1kcq,The Getaway,250386,False,1,0.797,0.642,0,-8.082,0.0725,0.079900,0.000012,0.3710,0.667,113.999,NaN
1,spotify:track:2oaK4JLVnmRGIO9ytBE1bt,Dark Necessities,302000,False,2,0.700,0.742,5,-6.777,0.0716,0.072200,0.019900,0.1100,0.197,91.959,NaN
2,spotify:track:0cv2LgkvEoQiGgFWcZaAMA,We Turn Red,200466,False,3,0.610,0.872,2,-5.428,0.0422,0.116000,0.147000,0.1520,0.842,164.038,NaN
3,spotify:track:0pjCkLjbgSLn5c0Ilwuv8z,The Longest Wave,211520,False,4,0.476,0.511,8,-6.250,0.0324,0.319000,0.000129,0.1190,0.339,148.158,NaN
4,spotify:track:2XTkpF9T2PKvcLgamGJGx1,Goodbye Angels,268733,False,5,0.365,0.804,9,-5.922,0.1050,0.104000,0.000066,0.1500,0.577,171.597,NaN
5,spotify:track:6GsP3uMCd0Dn5T37C93waZ,Sick Love,221440,False,6,0.591,0.794,4,-5.935,0.0690,0.183000,0.000000,0.2920,0.875,147.974,NaN
6,spotify:track:5m578MgxuEC956dulprCe8,Go Robot,263800,False,7,0.852,0.651,8,-6.254,0.0411,0.043200,0.019200,0.1580,0.960,131.010,NaN
7,spotify:track:0SMzXgUFqcAt0WNCsGm3tM,Feasting on the Flowers,202933,False,8,0.670,0.750,6,-5.502,0.0583,0.137000,0.000004,0.1970,0.880,82.041,NaN
8,spotify:track:5y7IfSDhBHFVnlGj1UrlA1,Detroit,226640,False,9,0.439,0.885,4,-4.679,0.0625,0.000277,0.003670,0.3720,0.495,92.738,NaN
9,spotify:track:3iXNy7VaY9OqJFBwzhmbOh,This Ticonderoga,215200,False,10,0.238,0.949,8,-4.575,0.2470,0.016000,0.000000,0.0864,0.405,87.843,NaN


In [74]:
# Scrape lyrics from a Genius.com song URL
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    lyrics = html.find('div', class_='lyrics').get_text()
    #remove identifiers like chorus, verse, etc
    lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
    #remove empty lines
    lyrics = os.linesep.join([s for s in lyrics.splitlines() if s])         
    return lyrics
# DEMO
print(scrape_song_lyrics('https://genius.com/Lana-del-rey-young-and-beautiful-lyrics'))

AttributeError: 'NoneType' object has no attribute 'get_text'

In [79]:
genius_token = config.genius['token']
genius_token

AttributeError: module 'config' has no attribute 'genius'